In [ ]:
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme="onedork", context="notebook", ticks=True, grid=True)

import numpy as np
import h5py
import os
import sys
import glob

import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
import pandas as pd
import tikzplotlib

# from tqdm import tqdm
from tqdm.notebook import tqdm_notebook as tqdm

import helper.circular
import fastpli.io
import fastpli.tools
import fastpli.objects
import fastpli.analysis

sim_path = "output/simulation/*.h5"
out_file = "output/analysis/"

In [ ]:
if False or not os.path.isfile(os.path.join(out_file, "pm.pkl")):
    os.makedirs("output/analysis", exist_ok=True)
    file_list = sorted(glob.glob(sim_path))

    # get resolution:
    with h5py.File(file_list[0], 'r') as h5f:
        for name in ['PM']:
            h5f_sub = h5f[name + '/p']
            res = h5f_sub['analysis/rofl/direction'].shape
    shape = (len(file_list), 2, res[0] * res[1])

    df = {'PM': pd.DataFrame(), 'LAP': pd.DataFrame()}

    for f, file in enumerate(tqdm(file_list)):
        with h5py.File(file, 'r') as h5f:
            for name in ['PM', 'LAP']:
                for m, model in enumerate(['p', 'r']):
                    h5f_sub = h5f[name + '/' + model]

                    epa_dir = h5f_sub['analysis/epa/0/direction'][...].ravel()
                    # epa_dir = helper.circular.remap(epa_dir, 0, np.pi)
                    epa_dir = helper.circular.center_mean(epa_dir, 0, np.pi)

                    rofl_dir = h5f_sub['analysis/rofl/direction'][...].ravel()
                    rofl_dir[rofl_dir > np.pi / 2] -= np.pi

                    df[name] = df[name].append(
                        {
                            "f0":
                            h5f_sub.attrs['parameter/f0_inc'],
                            "f1":
                            h5f_sub.attrs['parameter/f1_rot'],
                            "omega":
                            h5f_sub.attrs['parameter/omega'],
                            "psi":
                            h5f_sub.attrs['parameter/psi'],
                            "rofl_dir":
                            np.rad2deg(rofl_dir),
                            "rofl_inc":
                            h5f_sub['analysis/rofl/inclination'][...].ravel(),
                            "rofl_trel":
                            h5f_sub['analysis/rofl/t_rel'][...].ravel(),
                            "epa_trans":
                            h5f_sub['analysis/epa/0/transmittance'][
                                ...].ravel(),
                            "epa_dir":
                            np.rad2deg(epa_dir),
                            "epa_ret":
                            h5f_sub['analysis/epa/0/retardation'][...].ravel(),
                        },
                        ignore_index=True)

    df['PM'].to_pickle(os.path.join(out_file, "pm.pkl"))
    df['LAP'].to_pickle(os.path.join(out_file, "lap.pkl"))

df_pm = pd.read_pickle(os.path.join(out_file, "pm.pkl"))
df_lap = pd.read_pickle(os.path.join(out_file, "lap.pkl"))

In [ ]:
data = df_pm[(df_pm.f0 == 0.0) & (df_pm.f1 == 0.0)].explode("epa_dir")
grid = sns.FacetGrid(data,
                     col="omega",
                     hue="omega",
                     palette=sns.color_palette("dark", 10),
                     col_wrap=5,
                     height=2.5)
grid.map(plt.scatter, "psi", "epa_dir", marker="o")
grid.fig.tight_layout(w_pad=1)

In [ ]:
data = df_pm[(df_pm.f0 == 0.0) & (df_pm.f1 == 0.0)].explode("epa_dir")

fig, axes = plt.subplots(2, 5)
for i, omega in enumerate(data.omega.unique()):
    sns.boxplot(x="psi",
                y="epa_dir",
                data=data[data.omega == omega],
                orient='v',
                color=sns.color_palette("dark", 10)[i],
                ax=axes[i // 5, i % 5])

data = df_pm[(df_pm.f0 == 0.0) & (df_pm.f1 == 0.0)].explode("epa_ret")
fig, axes = plt.subplots(2, 5)
for i, omega in enumerate(data.omega.unique()):
    sns.boxplot(x="psi",
                y="epa_ret",
                data=data[data.omega == omega],
                orient='v',
                color=sns.color_palette("dark", 10)[i],
                ax=axes[i // 5, i % 5])

In [ ]:
skip = 1
for f0_inc in tqdm(df_pm.f0.unique()):
    for f1_rot in tqdm(df_pm[df_pm.f0 == f0_inc].f1.unique(), leave=False):
        hist_bin = lambda n_phi: np.linspace(
            0, np.pi, n_phi + 1, endpoint=True)

        epa_dir = df_pm[(df_pm.f0 == f0_inc)
                        & (df_pm.f1 == f1_rot)].explode("epa_dir")
        rofl_dir = df_pm[(df_pm.f0 == f0_inc)
                         & (df_pm.f1 == f1_rot)].explode("rofl_dir")

        fig, axs = plt.subplots(int(np.ceil(len(epa_dir.psi.unique()) / skip)),
                                int(np.ceil(
                                    len(epa_dir.omega.unique()) / skip)),
                                subplot_kw={'projection': 'polar'},
                                figsize=(30, 30))
        axs = np.atleast_2d(axs)

        for j, _ in enumerate(epa_dir.omega.unique()[::skip]):
            for i, _ in enumerate(epa_dir.psi.unique()[::skip]):
                axs[i, j].set_xticklabels([])
                axs[i, j].set_yticklabels([])

        for j, omega in enumerate(
                tqdm(epa_dir.omega.unique()[::skip], leave=False)):
            for i, psi in enumerate(
                    tqdm(epa_dir[epa_dir.omega == omega].psi.unique()[::skip],
                         leave=False)):

                data = epa_dir[(epa_dir.omega == omega)
                               & (epa_dir.psi == psi)].epa_dir.to_numpy(
                                   dtype=float)
                data = helper.circular.remap(np.deg2rad(data), np.pi, 0)
                h, x = np.histogram(data, hist_bin(72), density=True)
                x = x[:-1] + x[1] - x[0]
                x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
                h = np.append(np.concatenate((h, h), axis=0), h[0])
                axs[i, j].plot(x, h)
                h_max = np.max(h)

                # rofl
                data = rofl_dir[(epa_dir.omega == omega)
                                & (epa_dir.psi == psi)].rofl_dir.to_numpy(
                                    dtype=float)
                data = helper.circular.remap(np.deg2rad(data), np.pi, 0)
                h, x = np.histogram(data, hist_bin(72), density=True)
                x = x[:-1] + x[1] - x[0]
                x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
                h = np.append(np.concatenate((h, h), axis=0), h[0])
                axs[i, j].plot(x, h)
                h_max = np.max(h)

                # ground truth
                if not os.path.isfile(
                        f"../data/models/cube_2pop_psi_{psi:.1f}_omega_{omega:.1f}_.solved.pkl"
                ):
                    fbs = fastpli.io.fiber_bundles.load(
                        f"../data/models/cube_2pop_psi_{psi:.1f}_omega_{omega:.1f}_.solved.h5"
                    )
                    rot_inc = fastpli.tools.rotation.y(-np.deg2rad(f0_inc))
                    rot_phi = fastpli.tools.rotation.x(np.deg2rad(f1_rot))
                    rot = np.dot(rot_inc, rot_phi)
                    fbs = fastpli.objects.fiber_bundles.Rotate(fbs, rot)
                    fbs = fastpli.objects.fiber_bundles.Cut(
                        fbs, [[-30] * 3, [30] * 3])
                    phi, theta = fastpli.analysis.orientation.fiber_bundles(
                        fbs)
                    df_o = pd.DataFrame({"phi": phi, "theta": theta})
                    df_o.to_pickle(
                        f"../data/models/cube_2pop_psi_{psi:.1f}_omega_{omega:.1f}_.solved.pkl"
                    )
                else:
                    df_o = pd.read_pickle(
                        f"../data/models/cube_2pop_psi_{psi:.1f}_omega_{omega:.1f}_.solved.pkl"
                    )
                    phi = df_o.phi.to_numpy(float)
                    theta = df_o.phi.to_numpy(float)

                phi = helper.circular.remap(phi, np.pi, 0)

                h, x = np.histogram(phi, hist_bin(180), density=True)
                x = x[:-1] + x[1] - x[0]
                x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
                h = np.append(np.concatenate((h, h), axis=0), h[0])
                h = h / np.max(h) * h_max
                axs[i, j].plot(x, h)

        fig.savefig(f"phi_f0_{f0_inc}_f1_{f1_rot}.svg")